In [1]:
#William Dahl
#CSI 436
#Final Project
#May 9th, 2019

#This is a Regression model using the mode of a distrobution of given classes to predict an emoji

import numpy as np

#Breaks up the data into training and testing data as well as sperated the messages from the classes
def break_up_data():
    with open('data.txt', 'r') as data:
        i = 0
        
        #Makes the trainign data
        train = open('train_data.txt', 'w') #the messages to be used to train the model
        sentiment_train = open('sentiment_train.txt', 'w') #The classes of the messages int he triaing model
        
        #writes 80% of the messages to the training data
        #there are 7480 messages in total
        while i < 7480 * .8:
            line = data.readline()
            sentiment = line[1:28]
            message = line[30:]
            train.write(message.lower())
            sentiment_train.write(sentiment)
            sentiment_train.write(';')
            i += 1
        
        #gets last line
        #ensures there is not a empty space in the files
        line = data.readline()
        sentiment = line[1:28]
        message = line[30:len(line)-2]
        train.write(message.lower())
        sentiment_train.write(sentiment)
        i += 1
        
        #makes the testing data
        test = open('test_data.txt', 'w') #messages to be used for testing
        sentiment_test = open('sentiment_test.txt', 'w') #classes of the messages used in testing
        line = data.readline()
        
        #adds the remaing 20% of the data to the testing files
        while i < 7479:
            line = data.readline()
            sentiment = line[1:28]
            message = line[30:]
            test.write(message.lower())
            sentiment_test.write(sentiment)
            sentiment_test.write(';')
            line = data.readline()
            i += 1
        
        #gets last line to add to the testing file
        #ensures there is not a empty space in the files
        line = data.readline()
        sentiment = line[1:28]
        message = line[30:len(line)-2]
        test.write(message.lower())
        sentiment_test.write(sentiment)
        
#retrives the sentiment classes for the tringin data and stores it in the form of a matrix        
def get_sentiment_matrix():
    return np.matrix(open('sentiment_train.txt').read())

#Breaks the message into 1-grams to 4-grams
def create_n_grams(message):
    message_list = message.split()
    output = list()
    #loops through message
    for i in range(len(message_list)):
        #greates the 1-grams to 4-grams of the message
        for j in range(4):  
            gram = message_list[i]
            #makes the specifeid n-gram
            for k in range(j):
                if (i+k+1) < len(message_list):
                    gram += " " + message_list[i+k+1]
        
            #checks if the gram was already created by an earlier message
            if gram not in output:
                output.append(gram)
    
    return output

#determeins the overall class that each n-gram should belong too. 
def classify_n_grams():
    train_data = open('train_data.txt', 'r')#data
    sentiment_matrix = get_sentiment_matrix()#classes
    message = train_data.readline()
    classified_grams = dict()#our model
    i = 0
    
    #Creates the n-grams for the message 
    while len(message) > 1:
        grams = create_n_grams(message)
        #adds the class of the mesage to the list of classes for the n-gram
        for gram in grams:
            #adds the n-gram to the dictonary if it is not already in it
            if gram not in classified_grams:
                classified_grams[gram] = list()
            
            #adds the class to the value list of the n-gram
            for j in range(7):
                if sentiment_matrix[i, j] == 1:
                    classified_grams[gram].append(j)
        
        message = train_data.readline()
        i += 1

    #gets the Expected class value from the list of the classes in the file for the n-gram
    #sets the overall class for the n-gram to be the expected value of the class for the n-gram
    for gram in classified_grams:
        classified_grams[gram] = max(set(classified_grams[gram]), key=classified_grams[gram].count)
    
    return classified_grams

#classifes a given message based on the trained model classified_grams
def classify_message(message, classified_grams):
    grams = create_n_grams(message.lower()) #gets the n-grams from the given message
    gram_class = list() #distrobution of classes from each n-gram extracted from the message
    
    #loops through the n-grams extracted from the message
    for gram in grams:
        #if the n-gram is a key in our model
        if gram in classified_grams:
            #writes the class of the n-gram to the class ditrobution for the message
            for i in range(len(gram)):
                gram_class.append(classified_grams[gram])
                
    if len(gram_class) == 0:
        return 0
    
    #returns the class that appears the most in the ditrobution. 
    return max(set(gram_class), key=gram_class.count)

#Begein exectuing our model
#creates our model
classified_grams = classify_n_grams()

#loops trough the testing data to classify it
with open('test_data.txt', 'r') as test:
    #classes of the messages for testing
    S_matrix = np.matrix(open('sentiment_test.txt').read())
    wrong_count = 0 #keeps track of the wrongly classifed messages
    i = 0
    
    #classifys all of the messages
    for message in test:
        clas = classify_message(message, classified_grams)
        #checks of the class is wrong
        if S_matrix[i,clas] != 1:
            wrong_count += 1
        
        i += 1
     
    #acuraccy of the model
    print("test acuracy: ", 1-wrong_count/747)

test acuracy:  0.499330655957162
